In [6]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from qiskit.providers.fake_provider import FakeToronto  

import qiskit
from qiskit import IBMQ, QuantumCircuit, execute, assemble, schedule, transpile
# from qiskit.providers.ibmq.job import job_monitor
from qiskit.tools.visualization import plot_histogram

## Insert IBM Q account token here
# IBMQ.save_account('', overwrite=True)
# provider = IBMQ.load_account()
# ## Insert provider details
# provider = IBMQ.get_provider(hub='', group='', project='')
provider = FakeToronto()

## picked from other script
from numpy import pi
import scipy
import pickle
import itertools

import qiskit
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, IBMQ, Aer, execute
from qiskit.tools.monitor import job_monitor
from qiskit.compiler import transpile, assemble
from qiskit.visualization import plot_histogram, plot_circuit_layout
from qiskit.transpiler import PassManager
from qiskit.ignis.mitigation.measurement import (complete_meas_cal, CompleteMeasFitter)

import pprint
import copy

import networkx as nx

In [4]:
## load all my helper functions
import sys
import importlib
import compiler_helper_functions
importlib.reload(compiler_helper_functions)
from compiler_helper_functions import *
import helper_library
importlib.reload(helper_library)
from helper_library import *
import data_logging_functions
importlib.reload(data_logging_functions)
from data_logging_functions import *
import statistics
import eval_metrics
importlib.reload(eval_metrics)
from eval_metrics import *

In [5]:
## import edited version of Siddhartha's code functions
import cnot_circuit_skeleton_dd
importlib.reload(cnot_circuit_skeleton_dd)
from cnot_circuit_skeleton_dd import *

ModuleNotFoundError: No module named 'pandas'

In [ ]:
def generate_sequence_id_from_combination(combo,sequence_length,rename_list):
    
    bitstring = [0 for _ in range(sequence_length)]
    for i in combo:
        for remap_entry in rename_list:
            if i == remap_entry[0]: 
                bitstring[remap_entry[1]] = 1
                break
    bitstring.reverse()
    #print(bitstring)
    # reverse the bitstring
    sequence_string = ''
    for bit in bitstring:
        sequence_string = sequence_string + str(bit)
    sequence_id = convert_key_to_decimal(sequence_string,len(sequence_string))
    #print('Sequence String ', sequence_string, ' Sequence ID ', sequence_id)
    return sequence_string, sequence_id
    
    
## function to gather important gate counts from qobj
def gather_all_gate_counts_from_op_counts(qobj):
    op_counts = qobj.count_ops()
    op_keys = ['cx','sx','x','rz','delay','id']
    op_count_dict = {}
    for op in op_keys:
        if op not in list(op_counts.keys()):
            curr_op_counts = 0
        else:
            curr_op_counts = op_counts[op]
        op_count_dict[op] = curr_op_counts
        
    return op_count_dict                                                                                             
                         

In [ ]:
def run_all_experiments(program,machine):
    
    ## gather program name for metadata
    program_str = program[program.find('evaluation')+11: program.find('.qasm')]
    print('Program ', program_str)
    
    ## params 
    shots=8192
    recur_count=1 # for debugging
    repeats = 2 # use 16K shots for smaller benchmarks and 32K shots for the larger workloads
    program_qubits = num_qubits_from_qasm(program)
    if program_qubits>8:
        repeats =4
    
    #indices of qubits in the device
    backend,coupling_map,basis_gates = get_device_information(machine)
    # print(f'backend {backend}, coupling_map {coupling_map}')
    qubits = list(range(backend.configuration().to_dict()['n_qubits']))
    if len(qubits)>7:
        max_circuits_per_batch = 600
    else:
        max_circuits_per_batch = 75
    
    all_circuits = []
    all_gate_counts = []
    
    ## read the qasm and generate the quantum circuit
    qc = read_qasm(program)
    
    print('Ideal Simulation Step')
    print('Original Circuit Depth ', qc.depth())
   
    ## execute on ideal simulator 
    original_counts = execute_on_ideal_machine([qc],int(shots*repeats))
    
    ## compile 
    qc_out,qc_out_cx_cts,qc_out_op_cts = recursive_compile_noise_adaptive(qc,machine,recur_count=recur_count)
    qc_out = qc_out[0]
    #circuit_depth
    circuit_depth = qc_out.depth()
    
    #total qubit capacity of backend
    # device = provider.get_backend(machine)
    device = FakeToronto()
    qubits_in_device = len(device.properties().to_dict()['qubits'])

    #total number of classical bits used in the circuit
    num_clbits = qc_out.num_clbits

    #converting circuit to dag
    dag = circuit_to_dag(qc_out)
    
    #get the qubit indices from the DAG
    qubit_set = get_qubit_set(dag)
    
    #create a skeletal instruction table
    IDT_Frame=create_empty_InstructionTable(qubit_set=qubit_set, circuit_depth=circuit_depth+10)
    
    IDT_Frame_p=populate_InstructionTable(qubit_set,dag,IDT_Frame,mode='NotVisual')
    
    #apply zero filter to the created qubit evolution timestamp table
    IDT=zero_filter(IDT_Frame_p)
    
    ## determine the gate lengths and capture them in a dictionary
    cx_lengths,x_lengths,sx_lengths,id_lengths,rz_lengths = get_all_instruction_lengths(machine,provider)
    gate_lengths = {'cx': cx_lengths, 'sx': sx_lengths, 'id': id_lengths, 'rz': rz_lengths, 'x': x_lengths}
    
    #analog Instruction dependency table
    analog_IDT = adv_discrete_to_analog(IDT, gate_lengths, mode = 'NotVisual')
    
    # baseline circuit without DD
    total_shots = int(repeats*shots)
    baseline_circ = analog_IDT_to_circ(analog_IDT, gate_lengths, qubits_in_device, num_clbits, mode = 'normal')
    baseline_counts = execute_on_ideal_machine([baseline_circ],total_shots)
    all_circuits.append(baseline_circ)
    baseline_ops = gather_all_gate_counts_from_op_counts(baseline_circ)
    all_gate_counts.append(baseline_ops)
    
    # skeleton circuit without DD
    skeleton_circ = analog_IDT_to_skeleton_circ(analog_IDT, gate_lengths, qubits_in_device, num_clbits, mode = 'normal')
    skeleton_circ_counts = execute_on_ideal_machine([skeleton_circ],total_shots)
    all_circuits.append(skeleton_circ)
    skeleton_ops = gather_all_gate_counts_from_op_counts(skeleton_circ)
    all_gate_counts.append(skeleton_ops)
    
    ## find all possible DD combinations
    #all combinations is a list of all possible combinations of qubits that need to be considered for DD application
    qb = [int(i) for i in analog_IDT.columns]
    all_combinations = []
    rename_table = [(qb[i],i) for i in range(len(qb))]
    ## print(rename_table)--> useful to generate sequence ID
    
    sequence_strings, sequence_ids = [],[]
    for r in range(len(qb) + 1):
        combinations_object = itertools.combinations(qb, r)
        comb_list = list(combinations_object)
        all_combinations += comb_list
        for curr_combo in comb_list:
            seq_str, seq_id = generate_sequence_id_from_combination(curr_combo,len(qb),rename_table)
            sequence_strings.append(seq_str)
            sequence_ids.append(seq_id)
    #removing the first combination (DD on none) since it has already been added
    all_combinations = all_combinations[1:]
    #print(all_combinations)
    #creating all circuits
    print('Generating all DD sequences')
    for el in all_combinations:
    
        #creating the circuit for particular DD combination -- XYXY
        analog_IDT = adv_discrete_to_analog(IDT, gate_lengths, mode = 'NotVisual')
        circ = analog_IDT_to_circ(analog_IDT, gate_lengths, qubits_in_device, num_clbits, qubits_to_consider = list(el), mode = 'xyxy')
        all_circuits.append(circ)
        op_dicts = gather_all_gate_counts_from_op_counts(circ)
        all_gate_counts.append(op_dicts)
        
        ## Skeleton circuit for same DD combination - XYXY
        circ = analog_IDT_to_skeleton_circ(analog_IDT, gate_lengths, qubits_in_device, num_clbits, qubits_to_consider = list(el), mode = 'xyxy')
        all_circuits.append(circ)
        op_dicts = gather_all_gate_counts_from_op_counts(circ)
        all_gate_counts.append(op_dicts)
        
        ## Actual circuit for IBMQ-DD combination
        circ = analog_IDT_to_circ(analog_IDT, gate_lengths, qubits_in_device, num_clbits, qubits_to_consider = list(el), mode = 'ibmq_dd_delay')
        all_circuits.append(circ)
        op_dicts = gather_all_gate_counts_from_op_counts(circ)
        all_gate_counts.append(op_dicts)
        
        ## Skeleton circuit for IBMQ-DD combination
        circ = analog_IDT_to_skeleton_circ(analog_IDT, gate_lengths, qubits_in_device, num_clbits, qubits_to_consider = list(el), mode = 'ibmq_dd_delay')
        all_circuits.append(circ)
        op_dicts = gather_all_gate_counts_from_op_counts(circ)
        all_gate_counts.append(op_dicts)
    

                
    ## execute on the specific machine
    print('Real Machine Simulation Step')
    counts_vector, job_ids, job_noise_properties, job_time, job_results = execute_on_real_machine(all_circuits, shots=shots, machine_name=machine,max_acceptable_circuits_by_device=max_circuits_per_batch,repeats=repeats)
    ## debug 
    ##counts_vector = execute_on_ideal_machine(all_circuits, shots)
    
    ## Readjust the counts
    print('Readjusting All Dictionaries')
    data_dictionary = {}
    data_dictionary['program_name'] = program_str
    data_dictionary['post_compile_depth'] = circuit_depth
    data_dictionary['ideal_counts_baseline'] = original_counts[0]
    data_dictionary['ideal_counts_skeleton'] = skeleton_circ_counts[0]
    data_dictionary['all_dd_combinations'] = all_combinations
    data_dictionary['original_xyxy'] = {}
    data_dictionary['skeleton_xyxy'] = {}
    data_dictionary['original_ibmq_dd'] = {}
    data_dictionary['skeleton_ibmq_dd'] = {}
    
    ## insert the counts for no DD
    data_dictionary['original_xyxy'][0] = {'Sequence_id': sequence_ids[0], 'Sequence ': sequence_strings[0], 'counts': counts_vector[0], 'gate_cost': all_gate_counts[0]}
    data_dictionary['skeleton_xyxy'][0] = {'Sequence_id': sequence_ids[0], 'Sequence ': sequence_strings[0], 'counts': counts_vector[1], 'gate_cost': all_gate_counts[1]}
    data_dictionary['original_ibmq_dd'][0] = {'Sequence_id': sequence_ids[0], 'Sequence ': sequence_strings[0], 'counts': counts_vector[0], 'gate_cost': all_gate_counts[0]}
    data_dictionary['skeleton_ibmq_dd'][0] = {'Sequence_id': sequence_ids[0], 'Sequence ': sequence_strings[0], 'counts': counts_vector[1], 'gate_cost': all_gate_counts[1]}
    
    base_index = 2
    for dd_combination_id in range(1,len(all_combinations)+1): # shift by 1 to accomodate no dd case from getting overwritten
        data_dictionary['original_xyxy'][dd_combination_id] = {'Sequence_id': sequence_ids[dd_combination_id], 'Sequence ': sequence_strings[dd_combination_id], 'counts': counts_vector[base_index], 'gate_cost': all_gate_counts[base_index]}
        data_dictionary['skeleton_xyxy'][dd_combination_id] = {'Sequence_id': sequence_ids[dd_combination_id], 'Sequence ': sequence_strings[dd_combination_id], 'counts': counts_vector[base_index+1], 'gate_cost': all_gate_counts[base_index+1]}
        data_dictionary['original_ibmq_dd'][dd_combination_id] = {'Sequence_id': sequence_ids[dd_combination_id], 'Sequence ': sequence_strings[dd_combination_id], 'counts': counts_vector[base_index+2], 'gate_cost': all_gate_counts[base_index+2]}
        data_dictionary['skeleton_ibmq_dd'][dd_combination_id] = {'Sequence_id': sequence_ids[dd_combination_id], 'Sequence ': sequence_strings[dd_combination_id], 'counts': counts_vector[base_index+3], 'gate_cost': all_gate_counts[base_index+3]}
        ### move to the next set 
        base_index = base_index+4 
    #print(data_dictionary)
    # for debugging only return data_dictionary return all_circuits
         
    return data_dictionary, all_circuits, job_ids, job_noise_properties, job_time, job_results



In [ ]:
## run the simulations for a batch overnight

import glob
import os
import re

files = '../benchmarks/' + '*.qasm'
filelist =  glob.glob(files)
#print(filelist)
for program in filelist:
    print('Program ', program)
    # machine_name = 'ibmq_guadalupe'
    machine_name = 'fake_toronto'
    program_str = '_'+ program[program.find('benchmarks')+11: program.find('.qasm')] + '_'
    pattern = re.compile(machine_name+program_str)
    ## check if it is possible to even simulate the program at all (number of qubits in program <= number of qubits on the device)
    num_qubits_in_program = num_qubits_from_qasm(program)
    backend,_,_ = get_device_information(machine_name)
    num_qubits_on_device = backend.configuration().to_dict()['n_qubits']
    print('Num Qubits in Program ', num_qubits_in_program, ' Num Qubits on Device ', num_qubits_on_device)
    if num_qubits_in_program<=num_qubits_on_device:
        simulate_bit = 1
    else:
        simulate_bit = 0
    for filepath in os.listdir('../logfiles/'):
        if pattern.match(filepath):
            #do stuff with matching file
            print('File exists')
            simulate_bit = 0
            break
    if simulate_bit==1:
        print('Simulate')
        char_output, post_compile_circuits,job_ids, job_noise_properties, job_time, job_results  = run_all_experiments(program,machine_name)
        print(f'DEBUG Char_output {char_output}')
        currentDT = datetime.datetime.now()
        datestring = str(currentDT.month) + '_' + str(currentDT.day) + '_' + str(currentDT.hour)

        logname = '../logfiles/'+machine_name+program_str+datestring+'.log'
        picklename = '../pickle_dumps/'+machine_name+program_str+datestring+'.pkl'

        ## dump all the information into logfile and pickle file
        write_data_dictionary_into_logfile(char_output,outputfname=logname)
        dump_objects_into_pickle_file(picklename,post_compile_circuits,job_ids,job_noise_properties,job_time,job_results)

Program  ../benchmarks/QFT-6-A.qasm
Num Qubits in Program  6  Num Qubits on Device  27
Simulate
Program: rks/QFT-6-A
Ideal Simulation Step
Original Circuit Depth: 24
Circuit 0:
     ┌───┐                                                              »
q_0: ┤ H ├──────■───────────────────────────────────────────────────────»
     ├───┤      │                                                       »
q_1: ┤ H ├──────┼──────────■────────────────────────────────────────────»
     ├───┤      │          │                                            »
q_2: ┤ H ├──────┼──────────┼──────────■─────────────────────────────────»
     ├───┤      │          │          │                                 »
q_3: ┤ H ├──────┼──────────┼──────────┼─────────■───────────────────────»
     ├───┤      │          │          │         │                  ┌───┐»
q_4: ┤ H ├──────┼──────────┼──────────┼─────────┼─────────■────────┤ H ├»
     ├───┤┌───┐ │U1(π/32)  │U1(π/16)  │U1(π/8)  │U1(π/4)  │U1(π/2) └───┘»
q_5: ┤ H 

TypeError: get_all_instruction_lengths() missing 1 required positional argument: 'provider'